This section scans all Spotify "Extended Streaming History" files in the directory, combines them into a single dataset, and performs initial time-based formatting.

In [1]:
import pandas as pd
import glob
import json
import os

def merge_spotify_data():
    """
    Finds all Streaming_History_Audio files, combines them,
    and performs initial data cleaning and formatting.
    """
    # Step 1: List all relevant JSON files
    # We use a pattern to match your Spotify data export naming convention
    file_pattern = "Streaming_History_Audio_*.json"
    files = glob.glob(file_pattern)
    
    if not files:
        print("Error: No files found matching the pattern! Please ensure your JSON files are in the same directory.")
        return None

    print(f"Found {len(files)} files. Starting the merge process...")

    all_records = []

    # Step 2: Read each file and extend the master list
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                all_records.extend(data)
            except Exception as e:
                print(f"Error reading {file}: {e}")

    # Step 3: Create a Pandas DataFrame
    df = pd.DataFrame(all_records)

    # Step 4: Format Time Data
    # Convert 'ts' (timestamp) column to datetime objects for easy analysis
    df['ts'] = pd.to_datetime(df['ts'])
    
    # Add helper columns for detailed temporal analysis
    df['year'] = df['ts'].dt.year
    df['month'] = df['ts'].dt.month
    df['date_time_formatted'] = df['ts'].dt.strftime('%Y-%m-%d %H:%M')

    # Step 5: Initial Cleanup (Optional)
    # You can filter out tracks played for less than 30 seconds to remove accidental clicks:
    # df = df[df['ms_played'] > 30000]

    # Save the combined result as a CSV for backup or Excel use
    # 'utf-8-sig' ensures special characters are displayed correctly in Excel
    output_filename = "spotify_full_history_merged.csv"
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    print(f"Success! Combined data saved to '{output_filename}'.")
    return df

# Execute the merge
spotify_df = merge_spotify_data()

# Preview the data
if spotify_df is not None:
    print("\nPreview of the merged dataset:")
    display_columns = ['ts', 'master_metadata_track_name', 'master_metadata_album_artist_name']
    print(spotify_df[display_columns].head())

Found 8 files. Starting the merge process...
Success! Combined data saved to 'spotify_full_history_merged.csv'.

Preview of the merged dataset:
                         ts  master_metadata_track_name  \
0 2016-05-07 11:08:09+00:00                 I'm America   
1 2016-05-07 11:08:11+00:00               Coming Around   
2 2016-05-07 11:08:11+00:00  A Summer Song - Radio Edit   
3 2016-05-07 11:08:14+00:00                     Beggars   
4 2016-05-07 11:08:14+00:00                  Sensations   

  master_metadata_album_artist_name  
0                            Cilver  
1                        Andie Case  
2                 Conner Youngblood  
3                          Krewella  
4                            Elohim  


This script serves as the master coordinator. It automatically detects all Spotify JSON files in your folder, merges them, and generates a comprehensive "Life-Time Report" grouped by each year. It provides 4 distinct "Top 10" tables for every year found in your data.

In [2]:
import pandas as pd
import json
import glob

def generate_full_history_report(output_file="Spotify_Full_History_Report.txt"):
    """
    Automatically detects all Streaming_History JSON files, merges them,
    and creates a year-by-year Top 10 report for songs and artists.
    """
    # 1. Locate all relevant JSON files
    files = glob.glob("Streaming_History_Audio_*.json")
    if not files:
        print("Error: No Spotify JSON files found! Please check your directory.")
        return

    print(f"Detected {len(files)} files. Merging data for a full timeline analysis...")
    
    combined_records = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            combined_records.extend(json.load(f))
    
    # Create DataFrame and prepare time/cleaning filters
    df = pd.DataFrame(combined_records)
    df['ts'] = pd.to_datetime(df['ts'])
    df['year'] = df['ts'].dt.year
    df = df.dropna(subset=['master_metadata_track_name'])
    
    # Filter: Only keep tracks played for more than 10 seconds
    df = df[df['ms_played'] > 10000]

    # 2. Generate the comprehensive report
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"{'='*30} SPOTIFY ALL-TIME ANALYSIS (2016-2026) {'='*30}\n\n")
        
        # Process each year chronologically
        years = sorted(df['year'].unique())
        
        for year in years:
            year_df = df[df['year'] == year]
            total_min = int(year_df['ms_played'].sum() / (1000 * 60))
            total_plays = len(year_df)
            
            f.write(f"\n{'#'*25} YEAR: {year} {'#'*25}\n")
            f.write(f"Summary: {total_min:,} minutes listened | {total_plays:,} total plays\n")
            f.write(f"{'-'*60}\n")

            # Helper function to write formatted tables to the text file
            def write_table(title, data, metric_name, unit_label):
                f.write(f"\n[{title}]\n")
                for i, (index, row) in enumerate(data.iterrows(), 1):
                    # Handle multi-index (Song + Artist) or single index (Artist only)
                    name = " - ".join(index) if isinstance(index, tuple) else index
                    secondary_label = 'plays' if unit_label == 'min' else 'min'
                    f.write(f"{i}. {name} -> {row[metric_name]} {unit_label} ({row['other']} {secondary_label})\n")

            # A. Top 10 Songs by Duration
            songs_duration = year_df.groupby(['master_metadata_track_name', 'master_metadata_album_artist_name']).agg(
                main=('ms_played', lambda x: int(x.sum() / (1000 * 60))), other=('ms_played', 'count')
            ).sort_values(by='main', ascending=False).head(10)
            write_table(f"{year} - TOP 10 SONGS BY DURATION", songs_duration, 'main', 'min')

            # B. Top 10 Songs by Play Count
            songs_count = year_df.groupby(['master_metadata_track_name', 'master_metadata_album_artist_name']).agg(
                main=('ms_played', 'count'), other=('ms_played', lambda x: int(x.sum() / (1000 * 60)))
            ).sort_values(by='main', ascending=False).head(10)
            write_table(f"{year} - TOP 10 SONGS BY PLAY COUNT", songs_count, 'main', 'plays')

            # C. Top 10 Artists by Duration
            artists_duration = year_df.groupby('master_metadata_album_artist_name').agg(
                main=('ms_played', lambda x: int(x.sum() / (1000 * 60))), other=('ms_played', 'count')
            ).sort_values(by='main', ascending=False).head(10)
            write_table(f"{year} - TOP 10 ARTISTS BY DURATION", artists_duration, 'main', 'min')

            # D. Top 10 Artists by Play Count
            artists_count = year_df.groupby('master_metadata_album_artist_name').agg(
                main=('ms_played', 'count'), other=('ms_played', lambda x: int(x.sum() / (1000 * 60)))
            ).sort_values(by='main', ascending=False).head(10)
            write_table(f"{year} - TOP 10 ARTISTS BY PLAY COUNT", artists_count, 'main', 'plays')
            
            f.write("\n" + "="*70 + "\n")

    print(f"Awesome! Your yearly music biography has been saved to: {output_file}")

# Run the master analysis
generate_full_history_report()

Detected 8 files. Merging data for a full timeline analysis...
Awesome! Your yearly music biography has been saved to: Spotify_Full_History_Report.txt


This interactive script allows you to focus on a single artist. By typing an artist's name, it scans your entire history and generates a dedicated chronological report showing your top songs by that specific artist for every year.

In [4]:
import pandas as pd
import json
import glob

def generate_artist_specific_report(artist_name):
    """
    Scans all JSON files and creates a detailed year-by-year 
    report for a specific artist requested by the user.
    """
    # 1. Merge all JSON files
    files = glob.glob("Streaming_History_Audio_*.json")
    if not files:
        print("Error: JSON files not found! Ensure the files are in the correct directory.")
        return

    all_records = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            all_records.extend(json.load(f))
    
    df = pd.DataFrame(all_records)
    df['ts'] = pd.to_datetime(df['ts'])
    df['year'] = df['ts'].dt.year
    
    # 2. Filter data for the specific artist (Case-insensitive)
    # The column name in Spotify data is 'master_metadata_album_artist_name'
    artist_df = df[df['master_metadata_album_artist_name'].str.lower() == artist_name.lower()].copy()

    if artist_df.empty:
        print(f"Error: No data found for artist '{artist_name}'.")
        return

    # Create a safe filename for the report
    report_filename = f"{artist_name.replace(' ', '_')}_Deep_Dive_Report.txt"

    # 3. Write the report
    with open(report_filename, "w", encoding="utf-8") as f:
        f.write(f"{'='*40}\n")
        f.write(f" ARTIST ANALYSIS: {artist_name.upper()}\n")
        f.write(f"{'='*40}\n\n")

        total_min = int(artist_df['ms_played'].sum() / (1000 * 60))
        total_plays = len(artist_df)
        f.write(f"All-Time Total Listening: {total_min:,} minutes\n")
        f.write(f"All-Time Total Play Count: {total_plays:,} times\n")
        f.write(f"{'-'*40}\n")

        # Loop through each year the artist was listened to
        years = sorted(artist_df['year'].unique())
        for year in years:
            year_df = artist_df[artist_df['year'] == year]
            
            f.write(f"\n>>>> {year} YEARLY DATA <<<<\n")
            f.write(f"Yearly Summary: {int(year_df['ms_played'].sum() / (1000 * 60))} min | {len(year_df)} plays\n")
            
            # Analyze Top 10 songs for that artist in that specific year (by Duration)
            song_stats = year_df.groupby('master_metadata_track_name').agg(
                min=('ms_played', lambda x: int(x.sum() / (1000 * 60))),
                count=('ms_played', 'count')
            ).sort_values(by='min', ascending=False).head(10)

            f.write(f"\n[Top Songs of {year}]\n")
            for i, (song, row) in enumerate(song_stats.iterrows(), 1):
                f.write(f"{i}. {song} -> {row['min']} min ({row['count']} plays)\n")
            
            f.write("-" * 30 + "\n")

    print(f"Success! Deep dive report for {artist_name} created: {report_filename}")

# --- EXECUTION ---
# You can type the exact name of the artist you want to analyze
target_artist = input("Enter the artist name you want to analyze: ")
generate_artist_specific_report(target_artist)

Success! Deep dive report for ariana grande created: ariana_grande_Deep_Dive_Report.txt


This script creates a "Hall of Fame" report for your entire listening history (from 2016 to present). It identifies the Top 50 Songs based on play count and the Top 50 Artists based on total minutes listened. This is a great way to see which artists truly dominate your long-term listening habits.

In [5]:
import pandas as pd
import json
import glob

def generate_all_time_top_50_report(output_file="Spotify_All_Time_Top50.txt"):
    """
    Combines all history files and generates a dual-section report:
    1. Top 50 Songs by Play Count
    2. Top 50 Artists by Total Duration
    """
    # 1. Locate and merge all JSON files (Audio + Video)
    # We include both because some favorite tracks might have been consumed as videos
    file_pattern = "Streaming_History_Audio_*.json"
    files = glob.glob(file_pattern)
    
    if not files:
        print("Error: JSON files not found! Ensure your streaming history files are in the directory.")
        return

    print(f"Merging {len(files)} files for the All-Time Hall of Fame analysis...")
    
    all_data_list = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            all_data_list.extend(json.load(f))
    
    df = pd.DataFrame(all_data_list)
    
    # Cleaning: Remove null track names and filter out very short interactions (under 1 second)
    df = df.dropna(subset=['master_metadata_track_name'])
    df = df[df['ms_played'] > 1000]

    # --- ANALYSIS 1: TOP 50 SONGS (BY PLAY COUNT) ---
    # This shows which songs you clicked on most frequently
    top_50_songs = df.groupby(['master_metadata_track_name', 'master_metadata_album_artist_name']).agg(
        count=('ms_played', 'count'),
        minutes=('ms_played', lambda x: round(x.sum() / (1000 * 60), 1))
    ).sort_values(by='count', ascending=False).head(50)

    # --- ANALYSIS 2: TOP 50 ARTISTS (BY TOTAL DURATION) ---
    # This shows who you spent the most time listening to
    top_50_artists = df.groupby('master_metadata_album_artist_name').agg(
        minutes=('ms_played', lambda x: int(x.sum() / (1000 * 60))),
        count=('ms_played', 'count')
    ).sort_values(by='minutes', ascending=False).head(50)

    # 2. Write the Ultimate Report
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"{'='*65}\n")
        f.write(f"      SPOTIFY ALL-TIME HALL OF FAME (2016-2026) REPORT\n")
        f.write(f"{'='*65}\n\n")

        f.write(f">>> SECTION 1: TOP 50 SONGS BY FREQUENCY (PLAY COUNT)\n")
        f.write(f"{'(Which songs did you repeat the most? Total minutes shown in brackets)'}\n")
        f.write("-" * 65 + "\n")
        for i, ((song, artist), row) in enumerate(top_50_songs.iterrows(), 1):
            f.write(f"{i:2d}. {song:.<40} {artist} -> {row['count']} plays ({row['minutes']} min)\n")

        f.write("\n\n")
        f.write(f">>> SECTION 2: TOP 50 ARTISTS BY DURATION (TOTAL TIME)\n")
        f.write(f"{'(Who occupied most of your time? Total play counts shown in brackets)'}\n")
        f.write("-" * 65 + "\n")
        for i, (artist, row) in enumerate(top_50_artists.iterrows(), 1):
            f.write(f"{i:2d}. {artist:.<40} {row['minutes']:,} min ({row['count']:,} plays)\n")

    print(f"\nSuccess! All-time Top 50 report generated: {output_file}")

# Execute analysis
generate_all_time_top_50_report()

Merging 8 files for the All-Time Hall of Fame analysis...

Success! All-time Top 50 report generated: Spotify_All_Time_Top50.txt


This script acts as a data pipeline that transforms raw Spotify JSON files into a structured .csv dataset. It aggregates your entire listening history, calculating the total play count and total minutes for every unique song. This dataset is perfect for further custom analysis or for use in external tools like Excel and Google Sheets.

In [6]:
import pandas as pd
import json
import glob

def generate_full_history_csv(output_filename="Spotify_Full_History_Dataset.csv"):
    """
    Finds all Streaming_History JSON files, merges them, cleans the data,
    and exports a structured CSV with song names, artists, counts, and minutes.
    """
    # 1. Gather all Spotify JSON history files (Audio and Video)
    files = glob.glob("Streaming_History_*.json")
    
    if not files:
        print("Error: No JSON files found! Please ensure your history files are in this directory.")
        return

    print(f"Combining {len(files)} files into a master dataset...")
    
    combined_data = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                combined_data.extend(data)
            except Exception as e:
                print(f"Error reading {file}: {e}")
    
    # Create the master DataFrame
    df = pd.DataFrame(combined_data)
    
    # 2. Data Cleaning
    # Remove records without a track name (like podcast episodes or system errors)
    df = df.dropna(subset=['master_metadata_track_name'])
    
    # Filter out interactions shorter than 1 second (accidental clicks)
    df = df[df['ms_played'] > 1000]

    # 3. Aggregation and Grouping
    # Calculate Play Count (Frequency) and Total Duration (Minutes) per song
    dataset_df = df.groupby(['master_metadata_track_name', 'master_metadata_album_artist_name']).agg(
        Play_Count=('ms_played', 'count'),
        Total_Minutes=('ms_played', lambda x: round(x.sum() / (1000 * 60), 2))
    ).reset_index()

    # Rename columns for clarity
    dataset_df.columns = ['Song_Name', 'Artist_Name', 'Play_Count', 'Total_Minutes']

    # Sort by frequency (most played at the top)
    dataset_df = dataset_df.sort_values(by='Play_Count', ascending=False)

    # 4. Export to CSV
    # 'utf-8-sig' is used to ensure special characters (accents, non-latin alphabets) display correctly in Excel
    dataset_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\nProcessing Complete! A master list with {len(dataset_df)} unique songs has been created.")
    print(f"Dataset saved as: {output_filename}")

# Run the dataset generator
generate_full_history_csv()

Combining 9 files into a master dataset...

Processing Complete! A master list with 8022 unique songs has been created.
Dataset saved as: Spotify_Full_History_Dataset.csv


This script creates a secondary dataset by filtering out any track played for less than 20 seconds. By removing accidental clicks and skipped tracks, this analysis focuses on the music you actually engaged with. This "Cleaned" dataset is the foundation for calculating your Skip Rate and Loyalty Score.

In [7]:
import pandas as pd
import json
import glob

def generate_cleaned_history_csv(output_filename="Spotify_Dataset_All_Time_Cleaned.csv"):
    """
    Combines all Spotify JSON files but only includes tracks played for 
    more than 20 seconds to ensure high data quality.
    """
    # 1. Locate all JSON files
    files = glob.glob("Streaming_History_*.json")
    
    if not files:
        print("Error: No JSON files found! Please check your folder.")
        return

    print(f"Merging {len(files)} files for the cleaned dataset...")
    
    all_records = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                all_records.extend(data)
            except Exception as e:
                print(f"Error reading {file}: {e}")
    
    df = pd.DataFrame(all_records)
    
    # 2. Data Cleaning & High-Quality Filtering
    # Remove rows with empty track names
    df = df.dropna(subset=['master_metadata_track_name'])
    
    # Filter: Keep only tracks played for more than 20,000 ms (20 seconds)
    # This removes skipped tracks and accidental interactions.
    df = df[df['ms_played'] > 20000]

    # 3. Aggregation
    cleaned_csv_df = df.groupby(['master_metadata_track_name', 'master_metadata_album_artist_name']).agg(
        Play_Count=('ms_played', 'count'),
        Total_Minutes=('ms_played', lambda x: round(x.sum() / (1000 * 60), 2))
    ).reset_index()

    cleaned_csv_df.columns = ['Song_Name', 'Artist_Name', 'Play_Count', 'Total_Minutes']

    # 4. Sorting
    # Sort by actual play count (real engagement)
    cleaned_csv_df = cleaned_csv_df.sort_values(by='Play_Count', ascending=False)

    # 5. Export to CSV
    cleaned_csv_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\nCleanup Complete: Tracks under 20 seconds have been excluded.")
    print(f"Total {len(cleaned_csv_df)} unique 'meaningfully played' tracks saved.")
    print(f"File created: {output_filename}")

# Run the high-quality dataset generator
generate_cleaned_history_csv()

Merging 9 files for the cleaned dataset...

Cleanup Complete: Tracks under 20 seconds have been excluded.
Total 7079 unique 'meaningfully played' tracks saved.
File created: Spotify_Dataset_All_Time_Cleaned.csv


Once you have generated your cleaned dataset, you can use this script to search for specific songs. It calculates the Global Rank of the song based on your total listening history and provides detailed statistics including play count and total minutes spent listening to that specific track.

In [8]:
import pandas as pd

def search_track():
    """
    Loads the cleaned CSV dataset and allows the user to search for songs
    interactively to see their ranking and total stats.
    """
    # 1. Load the cleaned CSV file
    try:
        df = pd.read_csv("Spotify_Dataset_All_Time_Cleaned.csv")
    except FileNotFoundError:
        print("Error: 'Spotify_Dataset_All_Time_Cleaned.csv' not found. Please run the dataset generator first.")
        return

    # 2. Add a Global Rank column based on Play Count
    #method='min' ensures that tied counts get the same rank
    df['Global_Rank'] = df['Play_Count'].rank(method='min', ascending=False).astype(int)

    while True:
        query = input("\nEnter the song name you want to search for (Press 'q' to quit): ").strip()
        
        if query.lower() == 'q':
            print("Exiting search engine...")
            break
        
        # 3. Perform the search (Case-insensitive partial match)
        results = df[df['Song_Name'].str.contains(query, case=False, na=False)]

        if not results.empty:
            print(f"\nResults found for '{query}':")
            print("-" * 90)
            # Formatted column headers
            print(f"{'RANK':<6} | {'SONG NAME':<35} | {'ARTIST':<20} | {'PLAYS':<8} | {'MINUTES':<10}")
            print("-" * 90)
            
            for _, row in results.iterrows():
                # Truncate long names for clean display
                song_display = str(row['Song_Name'])[:35]
                artist_display = str(row['Artist_Name'])[:20]
                
                print(f"{row['Global_Rank']:<6} | {song_display:<35} | {artist_display:<20} | {row['Play_Count']:<8} | {row['Total_Minutes']:<10}")
        else:
            print(f"No results found for '{query}' in your listening history.")

# Initialize the search engine
if __name__ == "__main__":
    search_track()


Results found for 'break free':
------------------------------------------------------------------------------------------
RANK   | SONG NAME                           | ARTIST               | PLAYS    | MINUTES   
------------------------------------------------------------------------------------------
1      | Break Free                          | Ariana Grande        | 319      | 1006.11   
2503   | break free - live                   | Ariana Grande        | 7        | 17.68     
4312   | Break Free (feat. Zedd)             | Ariana Grande        | 1        | 3.61      
4312   | Break Free (Karaoke Version) [Origi | Zoom Karaoke         | 1        | 0.36      
4312   | I Want To Break Free                | Queen                | 1        | 2.26      
Exiting search engine...


This script identifies your "Stable Favorites." It compares the Raw Dataset (all attempts to play a song) with the Cleaned Dataset (meaningful listens over 20 seconds). If a song has a very small difference between these two datasets, it means you almost never skip that song. This is the ultimate metric for identifying the core pillars of your music taste.

In [9]:
import pandas as pd

def generate_loyalty_stability_report(raw_csv="Spotify_Full_History_Dataset.csv", 
                                     clean_csv="Spotify_Dataset_All_Time_Cleaned.csv",
                                     output_file="Loyalty_Stability_Report.txt"):
    """
    Compares raw and cleaned data to identify songs you rarely skip.
    Filters: Minimum 50 clean plays and a maximum difference (skip count) of 6.
    """
    try:
        # 1. Load both datasets
        df_raw = pd.read_csv(raw_csv)
        df_clean = pd.read_csv(clean_csv)

        # Rename columns to avoid confusion during merging
        df_raw = df_raw.rename(columns={'Play_Count': 'Raw_Plays'})
        df_clean = df_clean.rename(columns={'Play_Count': 'Clean_Plays'})

        # 2. Merge datasets on Song and Artist name
        merged = pd.merge(df_clean[['Song_Name', 'Artist_Name', 'Clean_Plays']], 
                          df_raw[['Song_Name', 'Artist_Name', 'Raw_Plays']], 
                          on=['Song_Name', 'Artist_Name'])

        # 3. Apply Loyalty Kriterleri
        # Difference = Total clicks minus meaningful listens (represents skip count)
        merged['Difference'] = merged['Raw_Plays'] - merged['Clean_Plays']
        
        # Filter: At least 50 meaningful plays and no more than 6 skips/accidental clicks
        stability_list = merged[(merged['Clean_Plays'] >= 50) & (merged['Difference'] <= 6)].copy()

        # Sort by stability (lowest difference first) then by play count
        stability_list = stability_list.sort_values(by=['Difference', 'Clean_Plays'], ascending=[True, False])

        # 4. Export the findings to a text file
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(f"{'='*80}\n")
            f.write(f"{'STABLE FAVORITES REPORT (LOYALTY ANALYSIS)':^80}\n")
            f.write(f"{'='*80}\n\n")
            f.write("Analysis Criteria:\n")
            f.write("- Meaningful Plays (Duration > 20s) must be at least 50\n")
            f.write("- Difference between Total Attempts and Real Plays must be <= 6\n")
            f.write(f"{'-'*80}\n\n")
            
            f.write(f"{'RANK':<5} | {'SONG NAME':<30} | {'ARTIST':<20} | {'CLEAN':<6} | {'DIFF':<5}\n")
            f.write(f"{'-'*80}\n")

            for i, (_, row) in enumerate(stability_list.iterrows(), 1):
                song_name = str(row['Song_Name'])[:30]
                artist_name = str(row['Artist_Name'])[:20]
                f.write(f"{i:<5} | {song_name:<30} | {artist_name:<20} | {row['Clean_Plays']:<6} | {row['Difference']:<5}\n")

            f.write(f"\n\nTOTAL {len(stability_list)} STABLE SONGS FOUND.\n")

        print(f"Loyalty analysis complete! Stable favorites saved to: {output_file}")

    except Exception as e:
        print(f"An error occurred during analysis: {e}")

# Generate the loyalty report
generate_loyalty_stability_report()

Loyalty analysis complete! Stable favorites saved to: Loyalty_Stability_Report.txt


This is the most advanced analysis in the toolkit. It calculates a Loyalty Ratio by dividing your total attempts to play a song by your "meaningful" listens (more than 20 seconds).

A Ratio of 1.00 means you never skipped that song.

A higher ratio indicates a high "Skip Rate," suggesting the song might be part of a playlist but often gets skipped. This report helps distinguish between "background noise" and your "true masterpieces."

In [10]:
import pandas as pd

def generate_skip_rate_report(raw_csv="Spotify_Full_History_Dataset.csv", 
                              clean_csv="Spotify_Dataset_All_Time_Cleaned.csv",
                              output_file="Loyalty_Skip_Rate_Report.txt"):
    """
    Calculates the ratio between total attempts and meaningful plays.
    Identifies tracks with the highest retention/loyalty.
    """
    try:
        # 1. Load both datasets
        df_raw = pd.read_csv(raw_csv)
        df_clean = pd.read_csv(clean_csv)

        # Rename columns to differentiate metrics
        df_raw = df_raw.rename(columns={'Play_Count': 'Total_Attempts'})
        df_clean = df_clean.rename(columns={'Play_Count': 'Clean_Plays'})

        # 2. Merge datasets
        merged = pd.merge(df_clean[['Song_Name', 'Artist_Name', 'Clean_Plays']], 
                          df_raw[['Song_Name', 'Artist_Name', 'Total_Attempts']], 
                          on=['Song_Name', 'Artist_Name'])

        # 3. Calculate Ratio and Difference
        # Ratio = Total Clicks / Meaningful Plays. 1.00 is a perfect score.
        merged['Loyalty_Ratio'] = round(merged['Total_Attempts'] / merged['Clean_Plays'], 2)
        merged['Difference'] = merged['Total_Attempts'] - merged['Clean_Plays']

        # 4. Apply Filters (Focusing on songs with at least 50 real plays)
        loyalty_list = merged[merged['Clean_Plays'] >= 50].copy()

        # SORTING: Primary sort by Loyalty Ratio (Ascending), secondary by play count
        loyalty_list = loyalty_list.sort_values(by=['Loyalty_Ratio', 'Clean_Plays'], 
                                                ascending=[True, False])

        # 5. Export to Text File
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(f"{'='*100}\n")
            f.write(f"{'LOYALTY & SKIP RATE ANALYSIS (2016-2026)':^100}\n")
            f.write(f"{'='*100}\n\n")
            f.write("Definitions:\n")
            f.write("- RATIO: Total attempts divided by meaningful plays. 1.00 = Perfect (Never skipped).\n")
            f.write("- Analysis limited to songs with more than 50 'meaningful' (20s+) plays.\n")
            f.write(f"{'-'*100}\n\n")
            
            f.write(f"{'RANK':<5} | {'SONG NAME':<30} | {'ARTIST':<20} | {'CLEAN':<6} | {'RATIO':<6} | {'DIFF':<5}\n")
            f.write(f"{'-'*100}\n")

            for i, (_, row) in enumerate(loyalty_list.iterrows(), 1):
                song_name = str(row['Song_Name'])[:30]
                artist_name = str(row['Artist_Name'])[:20]
                f.write(f"{i:<5} | {song_name:<30} | {artist_name:<20} | {row['Clean_Plays']:<6} | {row['Loyalty_Ratio']:<6} | {row['Difference']:<5}\n")

            f.write(f"\n\nTOTAL {len(loyalty_list)} ANALYZED TRACKS FOUND.\n")

        print(f"Analysis complete! Skip Rate report saved to: {output_file}")

    except Exception as e:
        print(f"An error occurred during ratio analysis: {e}")

# Generate the skip rate report
generate_skip_rate_report()

Analysis complete! Skip Rate report saved to: Loyalty_Skip_Rate_Report.txt
